In [ ]:
from random import choice
from random import uniform
from numpy.random import randint
import numpy as np
import pandas as pd
from scipy.stats import gmean
from DE import de
import warnings
warnings.filterwarnings('ignore')

## DE hp initialization

In [ ]:
def initialization():
    parameters = {}
    
    parameters["np"] = choice([10, 15, 20, 25])
    parameters["mutation"] = choice(["rand/1", "rand/2", "best/1", "current-to-best/1"])
    parameters["cr"] = round(uniform(0.5, 0.99), 4)
    parameters["f"] = round(uniform(0.1, 0.99), 2)
    parameters["g"] = choice([10, 15, 20, 25])

    return parameters

In [ ]:
def generate_population(n):
  population = []
  for _ in range(n):
    chromosome = initialization()
    population.append(chromosome)
  return population

In [ ]:
def fitness_evaluation(model):
  acc, _ = model.run_de()
  print(f"metrics:{acc}")
  return acc

In [ ]:
# Roulette wheel selection method
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["np"] = choice([parent1["np"], parent2["np"]])
  child1["g"] = choice([parent1["g"], parent2["g"]])
  child1["mutation"] = choice([parent1["mutation"], parent2["mutation"]])

  child2["np"] = choice([parent1["np"], parent2["np"]])
  child2["g"] = choice([parent1["g"], parent2["g"]])
  child2["mutation"] = choice([parent1["mutation"], parent2["mutation"]])

  child1["cr"] = parent2["cr"]
  child2["cr"] = parent1["cr"]

  child1["f"] = parent2["f"]
  child2["f"] = parent1["f"]

  return [child1, child2]

In [ ]:
def mutation(chromosome):
  flag = round(uniform(0.0, 0.99), 4)
  if flag < 0.5:
    chromosome["cr"] += uniform(0.1, 0.4)
    if chromosome["cr"] >= 1.0:
      chromosome["cr"] = min(uniform(0.5, 0.99), 0.9)
  return chromosome

In [ ]:
generations = 12
threshold = 90
num_pop = 10

population = generate_population(num_pop)
acc_best = []
par_total = []
acc_total = []
par_best = []
for generation in range(generations):

  population_fitness = []
  per1=[]
  for chromosome in population:
    f = chromosome["f"]
    cr = chromosome["cr"]
    mu = chromosome["mutation"]
    np = chromosome["np"]
    g = chromosome["g"]

    try:
      model = de.de(cr=cr, f=f, np=np, mu_stra=mu, gen=g, N = 30)
      acc = fitness_evaluation(model)
      par_total.append(chromosome)
      acc_total.append(acc)
      per1.append(chromosome)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,3))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)
  print(population_fitness)
  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover(parent1, parent2)
  child1 = mutation(children[0])
  child2 = mutation(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))
    max_pop = max(population_fitness)
    c1=population_fitness.index(max_pop)
    par_best.append(per1[c1])
    acc_best.append(max_pop)

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])

print(par_total)
print(acc_total)
print(par_best)
print(acc_best)